# Kurulum

In [59]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/

In [ ]:
%cd MyDrive/

In [ ]:
%cd Colab Notebooks/

In [ ]:
# Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 diye bir klasör oluşturduktan sonra o klasöre erişiyoruz.
# Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

# Görselleştirme İşlemleri


In [ ]:
print(df.groupby('kategori').size())

In [ ]:
print(df.loc[df['kategori'] == 'Kanun', 'madde_sayisi'].isna)

In [ ]:
print(df.loc[df['kategori'] == 'Kanun'])

In [ ]:
print(df['data_text'][0])

In [ ]:
print(df['baslik'][0])

In [ ]:
df['baslik'][0].replace('  ',' ')

In [ ]:
# data_text ve baslik alanlarını temizliyoruz.
satir, sutun = df.shape
for s in range(satir):
  df['baslik'][s].replace('  ',' ')
  df['data_text'][s].replace('  ',' ')
  df['baslik'][s].replace('\n','')
  df['data_text'][s].replace('\n','')
  df['baslik'][s].replace('\r','')
  df['data_text'][s].replace('\r','')
  df['data_text'][s].lower()
  df['baslik'][s].lower()

In [ ]:
df['baslik'][5]

# Özel Ayar İçin Kullanım

In [ ]:
# data_text alanının ilk satırında madde sayamaya çalışalım!

# 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
# Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
basliginBaslandigiYer = df['data_text'][95].find(df['baslik'][95][:10])
print('basliginBaslandigiYer:',basliginBaslandigiYer)
# Başlığın uzunluğunu hesaplıyoruz.
basliginUzunlugu = len(df['baslik'][95])
print('basliginUzunlugu:',basliginUzunlugu)
# Başlığın data_text alanında bittiği yeri hesaplıyoruz
basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
verininBuyuklugu = len(df['data_text'][95])
print('basliginBittigiYer:',basliginBittigiYer)
print('verininBuyuklugu:',verininBuyuklugu)
# Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
# İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
maddeSayisi = 0
geciciMaddeSayisi = 0 # Geçici Madde 1 gibi verilen ek madde
ekMaddeSayisi = 0 # Madde içinde 10/a gibi verilen ek madde
ekMaddeSayisi2 = 0 # Ek Madde 1 gibi verilen ek madde
# aramaya baslayacağımız yeri belirliyoruz.,
aramaYapilacakYerinBasi = basliginBittigiYer

while (aramaYapilacakYerinBasi<verininBuyuklugu):
  print('maddeSayisi:',maddeSayisi)
  bulunanSayi = ''
  alinanKesit = df['data_text'][95][aramaYapilacakYerinBasi:]
  maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
  print('maddeninBaslayabilecegiYer:',maddeninBaslayabilecegiYer)
  tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
  print('tireninBaslayabilecegiYer:',tireninBaslayabilecegiYer)
  if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
    maddeninBaslayabilecegiYer+=5
    while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
      maddeninBaslayabilecegiYer+=1
      bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
      print(bulunanSayiyiOlusturacakParca)
      if bulunanSayiyiOlusturacakParca in '0123456789':
        bulunanSayi+=bulunanSayiyiOlusturacakParca
      else:
            break
    if bulunanSayi!='':
      if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
        maddeSayisi += int(bulunanSayi)-maddeSayisi
      elif int(bulunanSayi)<geciciMaddeSayisi+2:
        geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
      elif int(bulunanSayi)<ekMaddeSayisi2+2:
        ekMaddeSayisi2 += int(bulunanSayi)-ekMaddeSayisi2
  else:
    ekMaddeninYeri = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('/')
    if (ekMaddeninYeri<=(tireninBaslayabilecegiYer-2)) and ((tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=12):
      ekMaddeSayisi += 1
  aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+ekMaddeSayisi2)
print(maddeSayisi)

# İşlenmeyen Maddeler için Kod (Yazılacak)

In [ ]:
# İşlenmemiş Maddeleri Saymak için bir fonksiyon oluşturalım!
def islenmemisMaddeler(ms,kayit):
  if df['data_text'][kayit].find('İŞLENMEYEN GEÇİCİ MADDE')!=-1:
    ms += 1
  return ms

print(islenmemisMaddeler(10,350))

# Kategoriye Özel İşleme

## Kanun

In [ ]:
# data_text alanın madde sayılarını kontrol edelim.
hataliSay = 0
belgeSay = 0
satir, sutun = df.shape
print(satir,sutun)

for ilgiliKayit in range(satir):
  if df['kategori'][ilgiliKayit]=='Kanun':
    belgeSay += 1
    # 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
    # Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
    basliginBaslandigiYer = df['data_text'][ilgiliKayit].find(df['baslik'][ilgiliKayit][:10])
    # Başlığın uzunluğunu hesaplıyoruz.
    basliginUzunlugu = len(df['baslik'][ilgiliKayit])
    # Başlığın data_text alanında bittiği yeri hesaplıyoruz
    basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])
    # Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    geciciMaddeSayisi = 0
    ekMaddeSayisi = 0
    AmaddeSayisi = 0
    # aramaya baslayacağımız yeri belirliyoruz.,
    aramaYapilacakYerinBasi = 0

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      eskiMaddeIndex = maddeninBaslayabilecegiYer
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789/ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
            break
        if bulunanSayi!='':
          try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>(maddeSayisi)):
              maddeSayisi += int(bulunanSayi)-maddeSayisi
              #print(f"{ilgiliKayit} Madde {bulunanSayi}")
              
            elif (int(bulunanSayi)<=(geciciMaddeSayisi+2)) and (int(bulunanSayi)>(geciciMaddeSayisi)):
              geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
              #print(f"{ilgiliKayit} Geçici Madde {bulunanSayi}")
            elif (int(bulunanSayi)<=(ekMaddeSayisi+2)) and (int(bulunanSayi)>(ekMaddeSayisi)):
              ekMaddeSayisi += int(bulunanSayi)-ekMaddeSayisi
              #print(f"{ilgiliKayit} Ek Geçici Madde {bulunanSayi}")
          except:
            print(f"{ilgiliKayit} Madde {bulunanSayi}") #ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ
            AmaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
            
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    eskiMaddeSayisi = maddeSayisi
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+AmaddeSayisi)
    #maddeSayisi=islenmemisMaddeler(maddeSayisi,ilgiliKayit)
    if int(df['madde_sayisi'][ilgiliKayit])!=int(maddeSayisi):
      print(ilgiliKayit,df['madde_sayisi'][ilgiliKayit],maddeSayisi,eskiMaddeSayisi,geciciMaddeSayisi,ekMaddeSayisi,AmaddeSayisi)
      hataliSay += 1
    else:
      pass
print('Toplam kontrol edilen belge sayısı:',belgeSay)
print('Toplam hatalı kayıt sayısı:',hataliSay)
print('Doğruluk Oranı:',(belgeSay-hataliSay)/belgeSay)

## Kanun Hükmünde Kararname

In [ ]:
# data_text alanın madde sayılarını kontrol edelim.
hataliSay = 0
belgeSay = 0
satir, sutun = df.shape
print(satir,sutun)

for ilgiliKayit in range(satir):
  if df['kategori'][ilgiliKayit]=='Kanun Hükmünde Kararname':
    belgeSay += 1
    # 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
    # Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
    basliginBaslandigiYer = df['data_text'][ilgiliKayit].find(df['baslik'][ilgiliKayit][:10])
    # Başlığın uzunluğunu hesaplıyoruz.
    basliginUzunlugu = len(df['baslik'][ilgiliKayit])
    # Başlığın data_text alanında bittiği yeri hesaplıyoruz
    basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])
    # Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    geciciMaddeSayisi = 0
    ekMaddeSayisi = 0
    AmaddeSayisi = 0
    # aramaya baslayacağımız yeri belirliyoruz.,
    aramaYapilacakYerinBasi = 0

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      eskiMaddeIndex = maddeninBaslayabilecegiYer
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789/ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
            break
        if bulunanSayi!='':
          try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>(maddeSayisi)):
              maddeSayisi += int(bulunanSayi)-maddeSayisi
              #print(f"{ilgiliKayit} Madde {bulunanSayi}")
              
            elif (int(bulunanSayi)<=(geciciMaddeSayisi+2)) and (int(bulunanSayi)>(geciciMaddeSayisi)):
              geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
              #print(f"{ilgiliKayit} Geçici Madde {bulunanSayi}")
            elif (int(bulunanSayi)<=(ekMaddeSayisi+2)) and (int(bulunanSayi)>(ekMaddeSayisi)):
              ekMaddeSayisi += int(bulunanSayi)-ekMaddeSayisi
              #print(f"{ilgiliKayit} Ek Geçici Madde {bulunanSayi}")
          except:
            #print(f"{ilgiliKayit} Madde {bulunanSayi}") #ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ
            AmaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
            
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    eskiMaddeSayisi = maddeSayisi
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+AmaddeSayisi)
    #maddeSayisi=islenmemisMaddeler(maddeSayisi,ilgiliKayit)
    if int(df['madde_sayisi'][ilgiliKayit])!=int(maddeSayisi):
      print(ilgiliKayit,df['madde_sayisi'][ilgiliKayit],maddeSayisi,eskiMaddeSayisi,geciciMaddeSayisi,ekMaddeSayisi,AmaddeSayisi)
      hataliSay += 1
    else:
      pass
print('Toplam kontrol edilen belge sayısı:',belgeSay)
print('Toplam hatalı kayıt sayısı:',hataliSay)
print('Doğruluk Oranı:',(belgeSay-hataliSay)/belgeSay)

## Cumhurbaşkanlığı Kararnamesi

In [ ]:
# data_text alanın madde sayılarını kontrol edelim.
hataliSay = 0
belgeSay = 0
satir, sutun = df.shape
print(satir,sutun)

for ilgiliKayit in range(satir):
  if df['kategori'][ilgiliKayit]=='Cumhurbaşkanlığı Kararnamesi':
    belgeSay += 1
    # 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
    # Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
    basliginBaslandigiYer = df['data_text'][ilgiliKayit].find(df['baslik'][ilgiliKayit][:10])
    # Başlığın uzunluğunu hesaplıyoruz.
    basliginUzunlugu = len(df['baslik'][ilgiliKayit])
    # Başlığın data_text alanında bittiği yeri hesaplıyoruz
    basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])
    # Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    geciciMaddeSayisi = 0
    ekMaddeSayisi = 0
    AmaddeSayisi = 0
    # aramaya baslayacağımız yeri belirliyoruz.,
    aramaYapilacakYerinBasi = 0

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      eskiMaddeIndex = maddeninBaslayabilecegiYer
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789/ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
            break
        if bulunanSayi!='':
          try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>(maddeSayisi)):
              maddeSayisi += int(bulunanSayi)-maddeSayisi
              #print(f"{ilgiliKayit} Madde {bulunanSayi}")
              
            elif (int(bulunanSayi)<=(geciciMaddeSayisi+2)) and (int(bulunanSayi)>(geciciMaddeSayisi)):
              geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
              #print(f"{ilgiliKayit} Geçici Madde {bulunanSayi}")
            elif (int(bulunanSayi)<=(ekMaddeSayisi+2)) and (int(bulunanSayi)>(ekMaddeSayisi)):
              ekMaddeSayisi += int(bulunanSayi)-ekMaddeSayisi
              #print(f"{ilgiliKayit} Ek Geçici Madde {bulunanSayi}")
          except:
            print(f"{ilgiliKayit} Madde {bulunanSayi}") #ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ
            AmaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
            
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    eskiMaddeSayisi = maddeSayisi
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+AmaddeSayisi)
    #maddeSayisi=islenmemisMaddeler(maddeSayisi,ilgiliKayit)
    if int(df['madde_sayisi'][ilgiliKayit])!=int(maddeSayisi):
      print(ilgiliKayit,df['madde_sayisi'][ilgiliKayit],maddeSayisi,eskiMaddeSayisi,geciciMaddeSayisi,ekMaddeSayisi,AmaddeSayisi)
      hataliSay += 1
    else:
      pass
print('Toplam kontrol edilen belge sayısı:',belgeSay)
print('Toplam hatalı kayıt sayısı:',hataliSay)
print('Doğruluk Oranı:',(belgeSay-hataliSay)/belgeSay)

## Tüzük

In [ ]:
# data_text alanın madde sayılarını kontrol edelim.
hataliSay = 0
belgeSay = 0
satir, sutun = df.shape
print(satir,sutun)

for ilgiliKayit in range(satir):
  if df['kategori'][ilgiliKayit]=='Tüzük':
    belgeSay += 1
    # 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
    # Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
    basliginBaslandigiYer = df['data_text'][ilgiliKayit].find(df['baslik'][ilgiliKayit][:10])
    # Başlığın uzunluğunu hesaplıyoruz.
    basliginUzunlugu = len(df['baslik'][ilgiliKayit])
    # Başlığın data_text alanında bittiği yeri hesaplıyoruz
    basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])
    # Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    geciciMaddeSayisi = 0
    ekMaddeSayisi = 0
    AmaddeSayisi = 0
    # aramaya baslayacağımız yeri belirliyoruz.,
    aramaYapilacakYerinBasi = 0

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      eskiMaddeIndex = maddeninBaslayabilecegiYer
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789/ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
            break
        if bulunanSayi!='':
          try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>(maddeSayisi)):
              maddeSayisi += int(bulunanSayi)-maddeSayisi
              #print(f"{ilgiliKayit} Madde {bulunanSayi}")
              
            elif (int(bulunanSayi)<=(geciciMaddeSayisi+2)) and (int(bulunanSayi)>(geciciMaddeSayisi)):
              geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
              #print(f"{ilgiliKayit} Geçici Madde {bulunanSayi}")
            elif (int(bulunanSayi)<=(ekMaddeSayisi+2)) and (int(bulunanSayi)>(ekMaddeSayisi)):
              ekMaddeSayisi += int(bulunanSayi)-ekMaddeSayisi
              #print(f"{ilgiliKayit} Ek Geçici Madde {bulunanSayi}")
          except:
            print(f"{ilgiliKayit} Madde {bulunanSayi}") #ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ
            AmaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
            
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    eskiMaddeSayisi = maddeSayisi
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+AmaddeSayisi)
    #maddeSayisi=islenmemisMaddeler(maddeSayisi,ilgiliKayit)
    if int(df['madde_sayisi'][ilgiliKayit])!=int(maddeSayisi):
      print(ilgiliKayit,df['madde_sayisi'][ilgiliKayit],maddeSayisi,eskiMaddeSayisi,geciciMaddeSayisi,ekMaddeSayisi,AmaddeSayisi)
      hataliSay += 1
    else:
      pass
print('Toplam kontrol edilen belge sayısı:',belgeSay)
print('Toplam hatalı kayıt sayısı:',hataliSay)
print('Doğruluk Oranı:',(belgeSay-hataliSay)/belgeSay)

## Yönetmelik

In [ ]:
# data_text alanın madde sayılarını kontrol edelim.
hataliSay = 0
belgeSay = 0
satir, sutun = df.shape
print(satir,sutun)

for ilgiliKayit in range(satir):
  if df['kategori'][ilgiliKayit]=='Yönetmelik':
    belgeSay += 1
    # 'baslik' alanından ilk 10 karakteri 'data_text' alanında aratıyoruz.
    # Resmi belgenin başlığının 'data_text' alanında hangi indiste başladığını bulmuş oluyoruz.
    basliginBaslandigiYer = df['data_text'][ilgiliKayit].find(df['baslik'][ilgiliKayit][:10])
    # Başlığın uzunluğunu hesaplıyoruz.
    basliginUzunlugu = len(df['baslik'][ilgiliKayit])
    # Başlığın data_text alanında bittiği yeri hesaplıyoruz
    basliginBittigiYer = basliginBaslandigiYer+basliginUzunlugu
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])
    # Başlığın bittiği yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    geciciMaddeSayisi = 0
    ekMaddeSayisi = 0
    AmaddeSayisi = 0
    # aramaya baslayacağımız yeri belirliyoruz.,
    aramaYapilacakYerinBasi = 0

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      eskiMaddeIndex = maddeninBaslayabilecegiYer
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789/ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
            break
        if bulunanSayi!='':
          try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>(maddeSayisi)):
              maddeSayisi += int(bulunanSayi)-maddeSayisi
              #print(f"{ilgiliKayit} Madde {bulunanSayi}")
              
            elif (int(bulunanSayi)<=(geciciMaddeSayisi+2)) and (int(bulunanSayi)>(geciciMaddeSayisi)):
              geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
              #print(f"{ilgiliKayit} Geçici Madde {bulunanSayi}")
            elif (int(bulunanSayi)<=(ekMaddeSayisi+2)) and (int(bulunanSayi)>(ekMaddeSayisi)):
              ekMaddeSayisi += int(bulunanSayi)-ekMaddeSayisi
              #print(f"{ilgiliKayit} Ek Geçici Madde {bulunanSayi}")
          except:
            print(f"{ilgiliKayit} Madde {bulunanSayi}") #ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ
            AmaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
            
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    eskiMaddeSayisi = maddeSayisi
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+AmaddeSayisi)
    #maddeSayisi=islenmemisMaddeler(maddeSayisi,ilgiliKayit)
    if int(df['madde_sayisi'][ilgiliKayit])!=int(maddeSayisi):
      print(ilgiliKayit,df['madde_sayisi'][ilgiliKayit],maddeSayisi,eskiMaddeSayisi,geciciMaddeSayisi,ekMaddeSayisi,AmaddeSayisi)
      hataliSay += 1
    else:
      pass
print('Toplam kontrol edilen belge sayısı:',belgeSay)
print('Toplam hatalı kayıt sayısı:',hataliSay)
print('Doğruluk Oranı:',(belgeSay-hataliSay)/belgeSay)

# Yapılacaklar

In [ ]:
# Uç caselerin sayılmasında kaldık